# Introduction to NumPy for Engineering Simulations

## 1. What is NumPy and why do we use it?

**NumPy** (Numerical Python) is a fundamental Python library for:

* numerical computation
* vector and matrix operations
* scientific and engineering simulations

In engineering problems (FEM, networks, CFD, heat transfer), we constantly work with:

* vectors of properties (lengths, diameters, stiffnesses)
* matrices (connectivity, stiffness matrices)
* element-wise formulas

NumPy allows us to write code that looks **very close to the mathematical formulation**.

---

### Why not use plain Python lists?

Python lists are **general-purpose containers**, but:

* they do not support vectorized math
* operations are slow for large systems
* matrix algebra is cumbersome

NumPy arrays are:

* **fast**
* **memory-efficient**
* **mathematically consistent**
* designed for **scientific computing**

---

## 2. NumPy arrays: the basic data structure

The core object in NumPy is the **array**.

### Creating an array

```python
import numpy as np

a = np.array([1, 2, 3])
```

This represents the mathematical vector:

$$
\mathbf{a} = [1, 2, 3]
$$

Key properties:

```python
a.shape      # (3,)
a.ndim       # 1
a.dtype      # data type (int, float, etc.)
```

---

## 3. Arrays vs scalars

A **scalar** is a single value:

```python
g = 9.81
nu = 1e-6
```

An **array** contains many values:

```python
d = np.array([0.4, 0.2, 0.283, 0.283, 0.573])
L = np.array([1000, 1000, 2000, 2000, 2000])
```

Mathematically:

$$
\mathbf{d} = [d_1, d_2, \dots, d_5], \quad
\mathbf{L} = [L_1, L_2, \dots, L_5]
$$

Each entry corresponds to **one element of the network**.

---

## 4. Element-wise operations (vectorization)

One of the most important concepts in NumPy is **element-wise operations**.

### Example: power operator

```python
d**4
```

This does **not** mean $(\sum d_i)^4$.

Instead, NumPy computes:

$$
[d_1^4, d_2^4, \dots, d_5^4]
$$

Each element is raised to the power independently.

---

### Example: multiplication of arrays

```python
d**4 / L
```

This computes:

$$
\left[
\frac{d_1^4}{L_1},
\frac{d_2^4}{L_2},
\dots
\right]
$$

This is called **element-wise division**.

> NumPy automatically matches elements by index.

---

## 5. Example from the fluid network code

Consider the expression:

```python
fluid.element_property = np.pi * g * d**4 / (128.0 * nu * L)
```

Let us rewrite it mathematically.

---

### Mathematical formulation

For each element $e$:

$$
k_e = \frac{\pi g}{128 \nu} \frac{d_e^4}{L_e}
$$

This is the **Hagen–Poiseuille conductivity**.

---

### How NumPy evaluates it

* `np.pi`, `g`, `128.0`, `nu` → **scalars**
* `d**4`, `L` → **arrays**

NumPy applies **broadcasting**:

1. Scalars are expanded to match the array size
2. Operations are applied element by element

So the result is an array:

```python
fluid.element_property.shape == (5,)
```

Each entry corresponds to one pipe.

---

## 6. Why this is powerful

Without NumPy, you would need loops:

```python
k = []
for i in range(len(d)):
    k.append(np.pi * g * d[i]**4 / (128.0 * nu * L[i]))
```

With NumPy:

```python
k = np.pi * g * d**4 / (128.0 * nu * L)
```

* shorter
* clearer
* faster
* closer to the equation

---

## 7. Creating matrices with NumPy

### Example: connectivity matrix

```python
connectivity = np.array([
    [0, 1],
    [0, 2],
    [1, 2],
    [1, 3],
    [2, 3]
])
```

This is a **2D array** (matrix):

```python
connectivity.shape   # (5, 2)
```

Mathematically:

$$
\text{connectivity} =
\begin{bmatrix}
0 & 1 \\
0 & 2 \\
1 & 2 \\
1 & 3 \\
2 & 3
\end{bmatrix}
$$

Each row represents **one element** connecting two nodes.

---

### Accessing matrix elements

```python
connectivity[0]      # [0, 1]
connectivity[0, 1]   # 1
connectivity[:, 0]   # all first-column entries
connectivity[:, 1]   # all second-column entries
```

---

## 8. Array slicing (very important)

**Slicing** means selecting parts of an array.

### 1D slicing

```python
d[0]        # first element
d[-1]       # last element
d[1:4]      # elements 1 to 3
d[:3]       # first three elements
```

Mathematically:

$$
d[1:4] = [d_2, d_3, d_4]
$$

---

### 2D slicing

```python
connectivity[:, 0]   # all rows, column 0
connectivity[1:4, :] # rows 1 to 3, all columns
```

This is essential for:

* extracting node indices
* assembling matrices
* looping over elements

---

## 9. Creating arrays of constants

### Example: constant stiffness

```python
k = np.ones(5) * 1000
```

This produces:

$$
[1000, 1000, 1000, 1000, 1000]
$$

Very common for:

* uniform material properties
* constant conductivity
* identical springs

---

### Using `np.full`

```python
heat_area = np.full(net.n_elements, 200)
```

This means:

> “Create an array of length `n_elements` where every value is 200.”

---

## 10. Array size consistency (critical concept)

When doing element-wise operations:

```python
d**4 / L
```

NumPy requires:

```python
d.shape == L.shape
```

Otherwise, the operation is **not physically meaningful**.

This enforces **dimensional consistency**, similar to engineering equations.

---

## 11. Summary: how NumPy fits the network framework

| Concept             | NumPy role             |
| ------------------- | ---------------------- |
| Element properties  | 1D arrays              |
| Connectivity        | 2D arrays              |
| Governing equations | Vectorized expressions |
| Assembly            | Matrix operations      |
| Physical laws       | Element-wise formulas  |

---

## 12. Key takeaways for engineering students

* NumPy arrays represent **physical vectors**
* Operations follow **mathematical notation**
* Element-wise operations replace loops



## 13. Working with NumPy and Python Functions in Network-Based Solvers

### 1. Why functions are essential in engineering codes

In small scripts, calculations can be written directly.
However, **engineering solvers** (FEM, hydraulic networks, heat transfer) require:

* reusable logic
* clear separation between physics and numerics
* modular design
* iterative solvers calling the same operation many times

For this reason, **everything is written as functions**.

In `netsystems`, functions:

* **always receive vectors**
* **always return vectors**
* never operate on a single element at a time

This mirrors the mathematical formulation:

$$
\mathbf{k} = f(\mathbf{x})
$$

---

### 2. Structure of a Python function

Consider the function:

```python
def calculate_nonlinear_fluid_property(network):
    system = network.systems["fluid"]

    # Current nodal heads (solution or estimate)
    heads = system.x

    # Connectivity indices
    i = network.connectivity[:, 0].astype(int)
    j = network.connectivity[:, 1].astype(int)

    # Head difference ΔH
    delta_h = heads[i] - heads[j]
    abs_delta_h = np.abs(delta_h) + 1e-12

    # Pipe properties
    diameters = system.element_diameter[:network.n_elements]
    C = getattr(network, "chw", 110.0)

    if not hasattr(network, "element_lengths"):
        network.calculate_element_lengths_from_coordinates()

    # Nonlinear conductivity
    k = (
        0.2784
        * C
        * diameters ** 2.63
        / network.element_lengths ** 0.54
        / abs_delta_h ** 0.46
    )

    return k
```

This means:

* `def` → we are defining a function
* `calculate_nonlinear_fluid_property` → function name
* `network` → input argument (an object containing all data)

#### Engineering interpretation

> “Given the current state of the network, compute the nonlinear fluid conductivity of each element.”

Input:

* network geometry
* nodal heads
* pipe properties

Output:

* a vector of element conductivities

---

### 3. Extracting the system from the network

```python
system = network.systems["fluid"]
```

The `network` object may contain multiple systems (fluid, heat, etc.).

Here:

* `"fluid"` refers to the hydraulic system
* `system` is now a shortcut to all fluid-related data

This avoids writing long expressions repeatedly.

---

### 4. Nodal unknowns as vectors

```python
heads = system.x
```

`system.x` is a NumPy array:

$$
\mathbf{h} =
\begin{bmatrix}
h_1 \\
h_2 \\
\vdots \\
h_n
\end{bmatrix}
$$

These are:

* hydraulic heads
* temperatures
* displacements

depending on the system type.

Important:

* `heads` is **not a single value**
* it contains the state of **all nodes**

---

### 5. Connectivity as index vectors

```python
i = network.connectivity[:, 0].astype(int)
j = network.connectivity[:, 1].astype(int)
```

#### What is `connectivity`?

It is a matrix like:

$$
\begin{bmatrix}
n_1 & n_2 \\
n_3 & n_4 \\
\vdots & \vdots
\end{bmatrix}
$$

Each row defines one element connecting two nodes.

---

#### What are `i` and `j`?

* `i` → vector of **start node indices**
* `j` → vector of **end node indices**

Example:

```python
connectivity = np.array([
    [0, 1],
    [0, 2],
    [1, 2]
])

i = [0, 0, 1]
j = [1, 2, 2]
```

This allows vectorized access to nodal values.

---

### 6. Computing head differences vectorially

```python
delta_h = heads[i] - heads[j]
```

This is one of the most important lines.

#### What NumPy does

* `heads[i]` → heads at node i of each element
* `heads[j]` → heads at node j of each element

Result:

$$
\Delta \mathbf{H} =
\begin{bmatrix}
h_{i_1} - h_{j_1} \\
h_{i_2} - h_{j_2} \\
\vdots
\end{bmatrix}
$$

This computes **head differences for all elements at once**.

---

#### Absolute value and numerical stabilization

```python
abs_delta_h = np.abs(delta_h) + 1e-12
```

Why?

* Flow laws involve powers of $|\Delta H|$
* Zero head difference causes division by zero
* `1e-12` avoids numerical singularities

Engineering interpretation:

> We prevent singular conductivities when the flow tends to zero.

---

### 7. Extracting element properties as vectors

```python
diameters = system.element_diameter[:network.n_elements]
```

This creates:

$$
\mathbf{d} = [d_1, d_2, \dots, d_m]
$$

Each entry corresponds to one pipe.

---

#### Default parameters using Python attributes

```python
C = getattr(network, "chw", 110.0)
```

Meaning:

* If `network.chw` exists → use it
* Otherwise → assume `C = 110`

This is typical in engineering software:

* parameters may be optional
* reasonable defaults are provided

---

### 8. Ensuring geometric consistency

```python
if not hasattr(network, "element_lengths"):
    network.calculate_element_lengths_from_coordinates()
```

This guarantees that:

* element lengths exist
* geometry is consistent
* solver does not fail

Important design principle:

> Functions should validate the data they need.

---

### 9. Nonlinear conductivity formulation

```python
k = (
    0.2784
    * C
    * diameters ** 2.63
    / network.element_lengths ** 0.54
    / abs_delta_h ** 0.46
)
```

#### Mathematical model (Hazen–Williams type)

For each element $e$:

$$
k_e =
0.2784 C \frac{d_e^{2.63}}{L_e^{0.54} |\Delta H_e|^{0.46}}
$$

This is a **nonlinear conductivity**, because:

* $k_e$ depends on $\Delta H_e$
* $\Delta H_e$ depends on the solution $\mathbf{h}$

---

#### NumPy interpretation

* all operations are **element-wise**
* no loops
* every symbol corresponds to a vector

Result:

```python
k.shape == (n_elements,)
```

---

### 10. Returning vectors from functions

```python
return k
```

This is critical.

The solver expects:

* a vector of element properties
* one value per element

This matches the mathematical structure:

$$
\mathbf{k} \in \mathbb{R}^{n_\text{elements}}
$$

---

### 11. Big picture: data flow

#### Input

* network object
* nodal solution vector $\mathbf{x}$
* geometry and parameters

#### Processing

* extract vectors
* apply physical law
* operate element-wise

#### Output

* vector $\mathbf{k}$

This is exactly how nonlinear FEM and network solvers work.

---

### 12. Key engineering lessons

* Functions operate on **entire systems**, not single elements
* NumPy replaces loops with vector algebra
* Connectivity maps nodes → elements
* Physical laws are written once and reused
* Code mirrors the governing equations

---

### 13. Mental model to keep

> “If you can write the equation in vector form, you can write the code in NumPy.”

